<a href="https://colab.research.google.com/github/Karlajack/Final_Capstone_Project/blob/main/Airline_Customer_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 #Impoting general libraries
import warnings
import itertools
import random
from scipy.stats import uniform

# Data Analysis Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# import missingno as msno

# Data Preparation, Modeling, and Evaluation (SKLearn Modules)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler,RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv('/content/customer_reviews_2024.csv')

In [ ]:
df.head(2)

,Transaction ID,Transaction timestamp,Survey completion,First Name,Last Name,Gender,Phone number,Tags,Start time(GMT),Time of last update(GMT),...,What impressed you about XX Airlinestoday?,What can XX Airlines improve on that will delight you next time?,What did you not like about XX Airlines today?,"How satisfied were you with the overall experience, from boarding to destination airport?","Would you like to receive updates from XX Airlines about offers, news, surveys, and any other communication?",flight_date,destination,origin,Please state your country of residence,Thank you for flying with XX Airlines. Please take a moment to give us feedback on your experience with us. Do you wish to continue?
0,52825240,1/3/2024 4:01,TIMEDOUT,NaN,NaN,NaN,2.54721E+11,NaN,1/3/2024 4:01,1/10/2024 0:22,...,-,-,-,-,-,1/1/2024,NBO,EBB,-,-
1,52825244,1/3/2024 4:01,TIMEDOUT,NaN,NaN,NaN,2.54788E+11,NaN,1/3/2024 4:01,1/10/2024 0:22,...,-,-,-,-,-,1/1/2024,NBO,EBB,-,-


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15948 entries, 0 to 15947
Data columns (total 22 columns):
 #   Column                                                                                                                                Non-Null Count  Dtype 
---  ------                                                                                                                                --------------  ----- 
 0   Transaction ID                                                                                                                        15948 non-null  int64 
 1   Transaction timestamp                                                                                                                 15948 non-null  object
 2   Survey completion                                                                                                                     15948 non-null  object
 3   First Name                                                                  

In [ ]:
df.isna().sum().sort_values(ascending=False)

,0
Gender,118782
Tags,116535
Last Name,97522
First Name,96053
What did you not like about Kenya Airways today?,3
What can Kenya Airways improve on that will delight you next time?,3
Please state your country of residence,2
What impressed you about Kenya Airways today?,2
"On a scale of 0 to 10 (0-Less Likely, 10-Very Likely), How likely are you to recommend Kenya Airways to a friend or colleague?",1
origin,1
